Workspace
====

The core element of a project is the ``Workspace``. The Workspace holds core information about the corresponding `geoh5` project and all entities present in the project. 

![Workspace](./images/Workspace.png)


In [7]:
from geoh5py.workspace import Workspace

# Create a new project named "my_project"
workspace = Workspace("assets/my_project.geoh5")

You can either open an existing project or create a new project by simply entering the desired file name:

Groups
=====

`Groups` are effectively containers for other entities, such as ``Objects`` (Points, Curve, Surface, etc.) and/or other `Groups`. Groups can be used to establish `parent-child` relationships or to store information about a collection of entities.  

By default, the parent of any new `Entity` is the Workspace ``RootGroup``.  It is the only entity in the ``Workspace`` without a parent. Users rarely have to interect with the ``Root`` group as it is mainly used to maintain the overall project hierarchy.

![Root](./images/Root.png)



In [8]:
from geoh5py.groups import ContainerGroup
group = ContainerGroup.create(workspace, name='myGroup')

At creation, `"myGroup"` is written to the project ``geoh5`` file and visible in the Analyst project tree.

![Groups](./images/Groups.png)

Any entity can be accessed by its `name` or `uid` (unique identifier):

In [9]:
workspace.get_entity("myGroup")[0], workspace.get_entity(group.uid)[0]

(<geoh5py.groups.container_group.ContainerGroup at 0x17376981708>,
 <geoh5py.groups.container_group.ContainerGroup at 0x17376981708>)

Objects
======

The user can easily add objects to the project (Points, Curve, Surface, BlockModel, etc).

Points
------

The Point object simply consists of list of vertices defining positions in 3D space.

In [3]:
from geoh5py.objects import Points
import numpy as np

# Generate a numpy array of xyz locations
n = 100
radius, theta = np.arange(n), np.linspace(0, np.pi*8, n)

x, y = radius * np.cos(theta), radius * np.sin(theta)
z = (x**2. + y**2.)**0.5
xyz = np.c_[x.ravel(), y.ravel(), z.ravel()] # Form a 2D array

# Create the Point object
points = Points.create(
    workspace,       # The target Workspace
    name="MyPoints", # Name given
    vertices=xyz
)

NameError: name 'workspace' is not defined

![points](./images/points.png){width="50%"}

Curve
=====

The ``Curve`` object, also commonly referred to as a polyline, consists of a collection of vertices connected by line segments. Curves are often used to define contours, survey lines or geological contacts. It is a sub-class of the ``Points`` object with the added `cells` property that defines the segments connecting `vertices`.

In [14]:
from geoh5py.objects import Curve

# Create the Curve object
curve = Curve.create(
    workspace,       # The target Workspace
    vertices=xyz
)

![line](./images/line.png){width="50%"}

By default, all vertices are connected following the order of the input `vertices`. Alternatively, the `cells` property can be modified such that parts of the curve are assigned with a different `line_id`.

In [16]:
# Split the curve into two parts
line_id = np.r_[np.arange(n/2), np.arange(n/2, n)]
curve.line_id = line_id
workspace.finalize()



AttributeError: can't set attribute

Surface
---------

Similar to curves, the ``Surface`` object is made up of a collection of connected `vertices` that forms a net of triangles.

In [15]:
from scipy.spatial import Delaunay
from geoh5py.objects import Surface

# Create a 2D Delaunay triangulation
surf = Delaunay(points.vertices[:, :2])

# Create the Surface object
surface = Surface.create(
    workspace, 
    vertices=points.vertices, # Add vertices
    cells=surf.simplices      # Define cells
)

![surface](./images/surface.png){width="50%"}

Data
====

The main role of an object is to hold and display data. Data can be associated to different parts of an object depending on the type. For example: